In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-14 17:23:59--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-14 17:23:59--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-14 17:24:00--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [2]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 124013 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u352-ga-1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headl

In [3]:
import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf

sc=pyspark.SparkContext()
# spark = SparkSession.builder.getOrCreate()

spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")\
.config('spark.executor.memory', '70g').config('spark.driver.memory', '50g').config('spark.memory.offHeap.enabled', True).config('spark.memory.offHeap.size', '16g').config('spark.driver.maxResultsize', '0').config("spark.sql.shuffle.partitions", '4')\
.getOrCreate()
sc = SparkContext.getOrCreate()

# spark = SparkSession \
#     .builder \
#     .config("spark.jars", "/path/to/gcs-connector-hadoop2-latest.jar") \
#     .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 200000)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

In [5]:
df = spark.read.csv("/content/drive/MyDrive/3-2 university/Big_data_process_and_application/team_project/input/heart_2020_cleaned.csv", header=True, inferSchema=True)
df

DataFrame[HeartDisease: string, BMI: double, Smoking: string, AlcoholDrinking: string, Stroke: string, PhysicalHealth: double, MentalHealth: double, DiffWalking: string, Sex: string, AgeCategory: string, Race: string, Diabetic: string, PhysicalActivity: string, GenHealth: string, SleepTime: double, Asthma: string, KidneyDisease: string, SkinCancer: string]

#데이터 전처리

In [6]:
# 스키마 정보 확인
df.printSchema()

root
 |-- HeartDisease: string (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- AlcoholDrinking: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- PhysicalHealth: double (nullable = true)
 |-- MentalHealth: double (nullable = true)
 |-- DiffWalking: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Diabetic: string (nullable = true)
 |-- PhysicalActivity: string (nullable = true)
 |-- GenHealth: string (nullable = true)
 |-- SleepTime: double (nullable = true)
 |-- Asthma: string (nullable = true)
 |-- KidneyDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)



In [7]:
# 중복된 값 제거

print("원본 데이터 개수: ({}, {})".format(df.count(), len(df.columns)))

df=df.distinct()

print("중복된 행 제거 후 데이터 개수: ({}, {})".format(df.count(), len(df.columns)))

원본 데이터 개수: (319795, 18)
중복된 행 제거 후 데이터 개수: (301717, 18)


## 데이터 인코딩, 이상치 처리, 스케일링

In [8]:
major_df = df.filter(df.HeartDisease == "No")
minor_df = df.filter(df.HeartDisease == "Yes")

# 데이터 비율 및 수
print("데이터 label 0인 경우 : {}".format(major_df.count()))
print("데이터 label 1인 경우 : {}".format(minor_df .count()))

데이터 label 0인 경우 : 274456
데이터 label 1인 경우 : 27261


In [9]:
# 컬럼 정리
str_columns = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']
num_columns = ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']

### Label 인코딩 (기본적인 Yes, no 경우의 categorical 변수는 0, 1로 인코딩)

In [10]:
# label 인코딩
from pyspark.ml.feature import StringIndexer 

indexed = df
for col in str_columns:
    indexer = StringIndexer(inputCol=col, outputCol=col + "_Index")
    indexed = indexer.fit(indexed).transform(indexed)
indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+----------

### Ordinal Encoding (나이, 건강상태, 당뇨병 강도는 순서가 있다고 판단 Ordinal 인코딩 진행)

In [11]:
# ordinal encoding
from pyspark.sql.functions import when
# ord_columns=['AgeCategory', 'Diabetic', 'GenHealth']
# age_map = {'18-24' : 20, '25-29' : 25, '30-34' : 30, '35-39' : 35, '40-44' : 40, '45-49' : 45, '50-54' : 50, '55-59' : 55, '60-64' : 60, '65-69' : 65, '70-74' : 70, '75-79' : 75, '80 or older' : 80}
# genhealth_map = {'Excellent' : 4, 'Very good' : 3, 'Good' : 2, 'Fair' : 1, 'Poor' : 0}
# diabetic_map={'Yes' : 2, 'Yes (during pregnancy)' : 2, 'No, borderline diabetes' : 1, 'No' : 0}

# Age
indexed=indexed.withColumn("AgeCategory_Index", when(indexed.AgeCategory=="18-24", 20)
  .when(indexed.AgeCategory=="25-29", 25)
  .when(indexed.AgeCategory=="30-34", 30)
  .when(indexed.AgeCategory=="35-39", 35)
  .when(indexed.AgeCategory=="40-44", 40)
  .when(indexed.AgeCategory=="45-49", 45)
  .when(indexed.AgeCategory=="50-54", 50)
  .when(indexed.AgeCategory=="55-59", 55)
  .when(indexed.AgeCategory=="60-64", 60)
  .when(indexed.AgeCategory=="65-69", 65)
  .when(indexed.AgeCategory=="70-74", 70)
  .when(indexed.AgeCategory=="75-79", 75)
  .otherwise(80))

# GenHealth
indexed=indexed.withColumn("GenHealth_Index", when(indexed.GenHealth=="Excellent", 4)
  .when(indexed.GenHealth=="Very good", 3)
  .when(indexed.GenHealth=="Good", 2)
  .when(indexed.GenHealth=="Fair", 1)
  .otherwise(0))

# Diabetic 2, 2, 1, 0으로 설정
indexed=indexed.withColumn("Diabetic_Index", when(indexed.Diabetic=="Yes", 2)
  .when(indexed.Diabetic=="Yes (during pregnancy)", 2)
  .when(indexed.Diabetic=="No, borderline diabetes", 1)
  .otherwise(0))

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-----

### Race 원핫 인코딩

In [12]:
# Race onehot 인코딩 -> 미미해서 feature 제거할 수도 있음
from pyspark.ml.feature import OneHotEncoder

indexer = StringIndexer(inputCols=['Race'], outputCols=['Race_Index(temp)'])
indexed = indexer.fit(indexed).transform(indexed)

ohe_encoder = OneHotEncoder(inputCols=['Race_Index(temp)'], outputCols=['Race_Index'], dropLast=True)
indexed = ohe_encoder.fit(indexed).transform(indexed)

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_Index|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+--

### 이상치 제거 및 변환

In [13]:
# BMI 80 이상과, 수면시간이 24시간은 말이 안되므로 제거

indexed=indexed.filter((indexed.BMI < 80) & (indexed.SleepTime != 24))

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_Index|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+--

In [14]:
# 도메인 지식 없이 데이터 단순 지우기는 버겁다 생각-> log transformation 이용해서 skew값 안정화


# BMI
indexed=indexed.withColumn("BMI_Log", F.log1p("BMI"))

# PhysicalHealth
indexed=indexed.withColumn("PhysicalHealth_Log", F.log1p("PhysicalHealth"))

# MentalHealth
indexed=indexed.withColumn("MentalHealth_Log", F.log1p("MentalHealth"))

# SleepTime
indexed=indexed.withColumn("SleepTime_Log", F.log1p("SleepTime"))

indexed.show()


+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+------------------+------------------+------------------+------------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_Index|           BMI_Log|PhysicalHealth_Log

In [15]:
# 이상치 제거 후 비율 재확인
major_df = indexed.filter(indexed.HeartDisease_Index == 0.0)
minor_df = indexed.filter(indexed.HeartDisease_Index == 1.0)

print("이상치 제거 후 데이터 개수: ({}, {})".format(indexed.count(), len(indexed.columns)))
print("데이터 label 0인 경우 : {}".format(major_df.count()))
print("데이터 label 1인 경우 : {}".format(minor_df.count()))

이상치 제거 후 데이터 개수: (301653, 37)
데이터 label 0인 경우 : 274401
데이터 label 1인 경우 : 27252


### 수치형 변수 standardScaler를 통한 스케일링

In [16]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

num_columns = ['BMI_Log', 'PhysicalHealth_Log', 'MentalHealth_Log', 'SleepTime_Log']

assembler = VectorAssembler(inputCols=num_columns, outputCol="num_feauture")
num_vec = assembler.transform(indexed)
scaler = StandardScaler(inputCol="num_feauture", outputCol="num_feauture_Index", withMean=True, withStd=True)  # True로 하면 평균은 0, std=1로 '표준정규분포'
indexed= scaler.fit(num_vec).transform(num_vec)

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_I

In [18]:
# 최종 사용 컬럼
new_label=indexed.columns[18]
new_use_variable=indexed.columns[19:]
new_use_variable.remove('Race_Index(temp)')
new_use_variable.remove('Race_Index') # 인종 사용할 거면 해당 라인 삭제하기!
new_use_variable.remove('num_feauture')
new_use_variable.remove('num_feauture_Index')

new_final_col=indexed.columns[18:]
new_final_col.remove('Race_Index(temp)')
new_final_col.remove('Race_Index') # 인종 사용할 거면 해당 라인 삭제하기!
new_final_col.remove('num_feauture')
new_final_col.remove('num_feauture_Index')


new_final_col

['HeartDisease_Index',
 'Smoking_Index',
 'AlcoholDrinking_Index',
 'Stroke_Index',
 'DiffWalking_Index',
 'Sex_Index',
 'PhysicalActivity_Index',
 'Asthma_Index',
 'KidneyDisease_Index',
 'SkinCancer_Index',
 'AgeCategory_Index',
 'GenHealth_Index',
 'Diabetic_Index',
 'BMI_Log',
 'PhysicalHealth_Log',
 'MentalHealth_Log',
 'SleepTime_Log']

In [19]:
# 기본적인 최종 데이터 추출

new_final_data=indexed.select(new_final_col)

new_final_data.show()

+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|           BMI_Log|PhysicalHealth_Log|  MentalHealth_Log|     SleepTime_Log|
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|               0.0|          0.0|                  0.0|         0.0|              0.0|      0.0|               

#데이터 분리 
(데이터 증강 및 삭제는 반드시 분리된 상태의 원본 train_data를 기준으로만 진행해야 함.)

In [21]:
# 원본 데이터 분리 -기준점

# train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)
train_data, test_data = new_final_data.randomSplit([0.8, 0.2], seed=42)


train_data.show()

+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|           BMI_Log|PhysicalHealth_Log|  MentalHealth_Log|     SleepTime_Log|
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|               0.0|          0.0|                  0.0|         0.0|              0.0|      0.0|               

#전처리된 csv파일 생성

In [22]:
train_data.toPandas().to_csv('/content/drive/MyDrive/3-2 university/Big_data_process_and_application/team_project/train_data.csv', index=False, encoding='CP949')
test_data.toPandas().to_csv('/content/drive/MyDrive/3-2 university/Big_data_process_and_application/team_project/test_data.csv', index=False, encoding='CP949')